# SOC Triage Lab – Notebook

This notebook is used to manually analyze SOC alerts from the `alerts/` folder.

**Workflow:**
1. Set the alert file you want to analyze.
2. Load and inspect the alert.
3. Use this notebook together with the corresponding report in `reports/`.

# Alert 001 — SSH Brute-Force Attempt with Successful Login

**Category:** Authentication attack / Brute-force  
**Source:** EDR  
**Goal:** Validate whether repeated failed SSH logins followed by a success indicate compromise.  
**What to check:** Source IP, user account, event frequency, successful login timing, device identity.


In [3]:
from pathlib import Path
import json
from pprint import pprint

# Set which alert to load
ALERT_FILE = Path("../alerts/alert_001_ssh_bruteforce.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'cloud_provider': 'Azure',
 'dst_host': 'webserver-01.example.internal',
 'event_count': 120,
 'id': 'ALERT-001',
 'log_snippet': ['Jan 24 12:15:03 webserver-01 sshd[1024]: Failed password for '
                 'root from 203.0.113.45 port 51234 ssh2',
                 'Jan 24 12:17:41 webserver-01 sshd[1056]: Failed password for '
                 'root from 203.0.113.45 port 51302 ssh2',
                 'Jan 24 12:19:22 webserver-01 sshd[1077]: Failed password for '
                 'root from 203.0.113.45 port 51388 ssh2',
                 'Jan 24 12:24:55 webserver-01 sshd[1099]: Accepted password '
                 'for root from 203.0.113.45 port 51410 ssh2',
                 'Jan 24 12:29:57 webserver-01 sshd[1113]: Session opened for '
                 'user root by (uid=0)'],
 'product': 'Azure VM / Linux auth logs',
 'severity': 'Medium',
 'source': 'SIEM',
 'src_ip': '203.0.113.45',
 'timestamps': {'detection_time': '2025-12-06T12:30:10Z',
                'first_seen': '2

## Triage Notes (ALERT-001)

**Short summary:**  
- 120 failed SSH attempts from a single external IP (`203.0.113.45`) targeting `root` on `webserver-01`
- Followed by a succesful root login.
- Strong indication of brute-force compromise.

**Key indicators:**  
- User / account:  root
- Source IP:  203.0.113.45 (external)
- Destination host:  webserver-01 (Azure VM)
- Time window:  12:15 -> 12:29 UTC
- Event count:  ~120 failed logins + 1 success

**Initial assessment:**  
- [ ] False Positive  
- [ ] Benign but risky  
- [x] True Positive  
- [ ] Needs more information  

**Severity:**  
- [ ] Low  
- [ ] Medium  
- [ ] High  
- [x] Critical  

**Decision (L1):**  
- [ ] Close  
- [x] Escalate to L2 / IR  
- [ ] Create follow-up task  

**Notes for L2:**  
- Confirmed unauthorized root login after repeated failures.
- Recommended immediate VM isolation, credential reset, and forensic review.

# Alert 002 — Impossible Travel Sign-In

**Category:** Identity anomaly / Account compromise  
**Source:** SIEM (Azure AD Sign-In Logs)  
**Goal:** Determine whether two sign-ins from different continents in minutes indicate stolen credentials.  
**What to check:** Locations, IPs, MFA use, device differences, time gap, risk flags.


In [4]:
from pathlib import Path
import json
from pprint import pprint

# Set which alert to load
ALERT_FILE = Path("../alerts/alert_002_impossible_travel.json")

with ALERT_FILE.open("r", encoding="utf-8") as f:
    alert = json.load(f)

pprint(alert)

{'correlation': {'estimated_distance_km': 6200,
                 'impossible_travel_flag': True,
                 'time_diff_minutes': 12},
 'id': 'ALERT-002',
 'log_snippet': ['Sign-in success: alice.werner@example.com from 198.51.100.23 '
                 '(Frankfurt, DE), device WIN10-LAPTOP-01, MFA required.',
                 'Sign-in success: alice.werner@example.com from 203.0.113.88 '
                 '(New York, US), device BROWSER-EDGE, MFA not required.',
                 'Risk event: Impossible travel detected for user '
                 'alice.werner@example.com (12 minutes between Germany and '
                 'United States).'],
 'product': 'Azure AD Sign-In Logs',
 'risk_info': {'risk_event_types': ['impossibleTravel'], 'risk_level': 'high'},
 'severity': 'High',
 'sign_in_events': [{'auth_method': 'Password + MFA',
                     'client_app': 'Outlook',
                     'device': 'WIN10-LAPTOP-01',
                     'event_id': 'aad-evt-1001',
          

## Triage Notes (ALERT-002)

**Short summary:**
- Same user (`alice.werner@example.com`) signs in from Germany and 12 minutes later from the United States.
- Second sign-in uses a different device and no MFA, unlike the first.
- Physical travel is impossible -> strong indication of credential theft or session hijacking.

**Key indicators:**
- User / account: alice.werner@example.com
- First location: Frankfurt, Germany (198.51.100.23) - MFA required
- Second location: New York, USA (203.0.113.88) - Password only
- Time window: 08:12 -> 08:24 UTC (12 minutes)
- Discance: ~6200 km (impossible)
- Risk flags: `impossibleTravel = true`, `risk_level = high`

**Initial assessment:**
- [ ] False Positive  
- [ ] Benign but risky  
- [x] True Positive  
- [ ] Needs more information  

**Severity:**  
- [ ] Low  
- [ ] Medium  
- [x] High  
- [ ] Critical  

**Decision (L1):**  
- [ ] Close  
- [x] Escalate to L2 / IR  
- [ ] Create follow-up task

**Notes for L2:**
- Likely credential compromise; user should be contacted to confirm legitimacy.
- Recommend forced sign-out across sessions and immediate password reset.
- Check for additional risky sign-ins, mailbox access, file actions, or MFA manipulation.
- Investigate wheter other accounts show similar impossible travel patterns.